In [1]:
// id : ex_5.1_5.3

trait Stream[+A] {
    
    def headOption: Option[A] = this match {
        case Empty => None
        case Cons(h, t) => Some(h())
    }
    
    def toList: List[A] = this match {
        case Empty => Nil
        case Cons(h, t) => h() :: t().toList
    }
    
    def foldRight[B](z: => B)(f: (A, => B) => B): B = this match {
        case Cons(h, t) => f(h(), t().foldRight(z)(f))
        case _ => z
    }
    
    def exists(p: A => Boolean): Boolean =
        foldRight(false)((a, b) => p(a) || b)
    
    // ex 5.1
    def take(n : Int) : Stream[A] = {
        if( n <= 0) {
            Empty
        } else {
            this match {
                case Empty => Empty
                case Cons(h, t) => Cons(h, () => t().take(n-1))
            }  
        }
    }

    // ex 5.2
    def drop(n : Int) : Stream[A] = {
        if( n <= 0) {
            this
        } else {
            this match {
                case Empty => Empty
                case Cons(h, t) => t().drop(n-1)
            }  
        }
    }
    
    // ex 5.3
    def takeWhile(p: A => Boolean) : Stream[A] = {
        this match {
            case Empty => Empty
            case Cons(h, t) => if (p(h())) {
                Cons(h, () => t().takeWhile(p))
            } else {
                Empty
            }
        }
    }
    
    // ex 5.4
    def forAll(p: A => Boolean): Boolean = {
        this match {
            case Empty => true
            case Cons(h, t) => if (!p(h())) {
                false
            } else {
                t().forAll(p)
            }
        }
    }
    
    // ex 5.5 Implement takeWhile with FoldRight
    // foldRight[B](z: => B)(f: (A, => B) => B): B
    // f(h(), t().foldRight(z)(f))
    def takeWhileFR(p: A => Boolean) : Stream[A] = 
        foldRight(Empty.asInstanceOf[Stream[A]])((h, t) => if (p(h)) { 
            Cons(() => h, () => t) 
        } else {
            Empty
        })
    
    // ex 5.6 Implement headOption using FoldRight
    def headOptionFR: Option[A] = this.foldRight(None.asInstanceOf[Option[A]])((h, _) => Some(h))
    
    // ex 5.7 Implment map, filter append and flatMap using foldRight.
    // append should be non-strict in its argument.
    def map[B](f: A => B): Stream[B] = 
        this.foldRight(Empty.asInstanceOf[Stream[B]])((h, t) => Cons(() => f(h), () => t))
    
    def filter[B](p: A => Boolean): Stream[A] =
        this.foldRight(Empty.asInstanceOf[Stream[A]])((h, t) => {
            if (p(h)) {
                Cons(() => h, () => t)
            } else {
                t
            }
        })
    
    def append[B >: A](a2 : Stream[B]) : Stream[B] = 
        this.foldRight(a2)((h, t) => {
            Cons(() => h, () => t)
        })
    
    def flatMap[B](f: A => Stream[B]) : Stream[B] =
        this.foldRight(Empty.asInstanceOf[Stream[B]])((h, t) => {
            f(h).append(t)
        })
}

case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A]

object Stream {
    def cons[A](hd: => A, tl: => Stream[A]): Stream [A] = {
        lazy val head = hd
        lazy val tail = tl
        Cons(() => head, () => tail)
    }
    
    def empty[A]: Stream[A] = Empty
    
    def apply[A](as: A*): Stream[A] = 
        if (as.isEmpty) empty else cons(as.head, apply(as.tail: _*))
}


$line15.$read$$iw$$iw$Stream$@1c0ce04a

In [2]:
// id : try ex_5.1_5.3
println("headOption", Stream().headOption)
println("headOption", Stream(1,2,3).headOption)
println("take(2)",  Stream(1,2,3,4,5,6).take(2).toList)
println("drop(4)",  Stream(1,2,3,4,5,6).drop(4).toList)
println("takeWhile(x < 5)",  Stream(1,2,3,4,5,6).takeWhile((x : Int) => x < 5).toList)
println("forAll(x>0)", Stream(1,2,3,4,5,6).forAll((x : Int) => x > 0))
println("forAll(x>0)", Stream(1,2,3,4,5,6,-1).forAll((x : Int) => x > 0))

(headOption,None)
(headOption,Some(1))
(take(2),List(1, 2))
(drop(4),List(5, 6))
(takeWhile(x < 5),List(1, 2, 3, 4))
(forAll(x>0),true)
(forAll(x>0),false)


null

In [3]:
// id : try ex_5.5
println("takeWhileFR(x < 5)",  Stream(1,2,3,4,5,6).takeWhileFR((x : Int) => x < 5).toList)
println("takeWhileFR(x < 5)",  Stream().takeWhileFR((x : Int) => x < 5).toList)
println("takeWhileFR(x < 5)",  Stream(4).takeWhileFR((x : Int) => x < 5).toList)
println("takeWhileFR(x < 5)",  Stream(6).takeWhileFR((x : Int) => x < 5).toList)

(takeWhileFR(x < 5),List(1, 2, 3, 4))
(takeWhileFR(x < 5),List())
(takeWhileFR(x < 5),List(4))
(takeWhileFR(x < 5),List())


null

In [4]:
// id : try ex_5.6
println("headOptionFR", Stream().headOptionFR)
println("headOptionFR", Stream(1,2,3).headOptionFR)

(headOptionFR,None)
(headOptionFR,Some(1))


null

In [5]:
// id : try ex_5.7
println("map", Stream(1,2,3,4,5).map(x => "<" + x.toString + ">").toList)
println("filter", Stream(1,2,3,4,5).filter(x => x > 3).toList)
println("append", Stream(1,2).append(Stream(3,4)).toList)
println("flatMap", Stream(1,2,3).flatMap(x => Stream(x,x)).toList)

(map,List(<1>, <2>, <3>, <4>, <5>))
(filter,List(4, 5))
(append,List(1, 2, 3, 4))
(flatMap,List(1, 1, 2, 2, 3, 3))


null

In [6]:
// id : ex_5.8
def constant[A](a : A) : Stream[A] = Cons(() => a, () => constant(a))

constant: [A](a: A)Stream[A]


In [7]:
// id : try ex_5.8
constant("A").take(7).toList

[[A, A, A, A, A, A, A]]

In [8]:
// id : ex_5.9
def from(n : Int) : Stream[Int] = Cons(() => n, () => from(n+1))

from: (n: Int)Stream[Int]


In [9]:
// id : try ex_5.9
from(3).take(7).toList

[[3, 4, 5, 6, 7, 8, 9]]

In [10]:
// id : ex_5.10
def fibHelper(n : Int, n1: Int) : Stream[Int] = {
    if (n == 0) {
        Stream(0,1).append(fibHelper(1,1))
    } else {
        Cons(() => n + n1, () => fibHelper(n1, n + n1))
    }
}

def fibs() : Stream[Int] = fibHelper(0, 1)

Cons(<function0>,<function0>)

In [11]:
// id : try ex_5.10
fibs.take(10).toList

[[0, 1, 2, 3, 5, 8, 13, 21, 34, 55]]

In [12]:
// id : ex_5.11

def unfold[A, S](z: S)(f: S => Option[(A, S)]): Stream[A] = {
    f(z) match {
        case Some((a, s)) => Cons(() => a, () => unfold(s)(f))
        case None => Empty
    }
}

unfold: [A, S](z: S)(f: S => Option[(A, S)])Stream[A]


In [13]:
// id: ex_5.12

def fibsIter = (pn: (Int, Int)) => Some((pn._1, pn._2), (pn._2, pn._1+pn._2))
def unfoldFibs = Cons(() => 0,
                      () => unfold((0,1))(fibsIter).map(_._2))

def unfoldConstant[A](a : A) = unfold(a)(a => Some((a, a)))

def unfoldOnes = unfoldConstant(1)

def unfoldFrom(n: Int) : Stream[Int] = unfold(n)(n => Some(n, n+1))


fibsIter: ((Int, Int)) => Some[((Int, Int), (Int, Int))]
unfoldFibs: Cons[Int]
unfoldConstant: [A](a: A)Stream[A]
unfoldOnes: Stream[Int]
unfoldFrom: (n: Int)Stream[Int]


In [14]:
// id: try ex_5.12

println("unfoldFibs", unfoldFibs.take(12).toList)
println("unfoldConstant", unfoldConstant(3).take(10).toList)
println("unfoldFrom", unfoldFrom(4).take(10).toList)
println("unfoldOnes", unfoldOnes.take(10).toList)

(unfoldFibs,List(0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89))
(unfoldConstant,List(3, 3, 3, 3, 3, 3, 3, 3, 3, 3))
(unfoldFrom,List(4, 5, 6, 7, 8, 9, 10, 11, 12, 13))
(unfoldOnes,List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1))


null

In [15]:
// id : ex_5.13

def zipWith[A, B, C](as : Stream[A], bs: Stream[B])(f : (A, B) => C) : Stream[C] =
    unfold((as, bs))((ab : Tuple2[Stream[A], Stream[B]]) => {
    val a = ab._1
    val b = ab._2
    val result = for {
        ah <- a.headOption
        bh <- b.headOption
    } yield (f(ah, bh), (a.drop(1), b.drop(1)))
    result
})
                     

zipWith: [A, B, C](as: Stream[A], bs: Stream[B])(f: (A, B) => C)Stream[C]


In [16]:
// id : try ex_5.13

def add_zipWith(as : Stream[Int], bs : Stream[Int]) = zipWith(as, bs)((a : Int, b : Int) => a + b)
println(add_zipWith(Stream(1,2,3), Stream(1,2,3)).toList)
println(add_zipWith(Stream(), Stream(1,2,3)).toList)
println(add_zipWith(Stream(1,8), Stream(3,4,5)).toList)

List(2, 4, 6)
List()
List(4, 12)


null